In [60]:
# Install packages if necessary
# !pip install numpy pandas plotly fastparquet kaleido

In [61]:
# auto reload modules for jupyter notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
# import library
import pandas as pd
import plotly.graph_objs as go
import numpy as np

# import order of features
from order import feature_order, cross_cutting_order, mapping, apps_2024

In [63]:
# read in data
df = pd.read_parquet('../data/raw/features.parquet')

# remove travel domain
df = df.loc[:, ~df.columns.get_level_values('domain').str.contains('Travel')]

# filter apps: drop 2024
df_filtered = df.drop(columns=[app for app in df.columns.get_level_values('app') if app in apps_2024], level='app')

# make sub data frames for phase-specific and cross-cutting features
# create a dataframe for cross-cutting features
df_cross_cutting = df_filtered.loc[df_filtered.index.get_level_values('type') == 'Cross-Cutting']
df_cross_cutting.head(5)

domain                                                           Finance  \
app                                             52 Weeks Money Challenge   
type          phase            feat   subfeat                              
Cross-Cutting Eliciting        Social Relations                        0   
              Defining         Social Relations                        0   
              Setting          Social Relations                        0   
              Operationalizing Social Relations                        0   
              Pursuing         Social Relations                        0   

domain                                                                          \
app                                             Bills Reminder, Budget Planner   
type          phase            feat   subfeat                                    
Cross-Cutting Eliciting        Social Relations                              0   
              Defining         Social Relations                              0   
              Setting          Social Relations                              0   
              Operationalizing Social Relations                              0   
              Pursuing         Social Relations                              0   

domain                                                                        \
app                                             Fortune City - A Finance App   
type          phase            feat   subfeat                                  
Cross-Cutting Eliciting        Social Relations                            0   
              Defining         Social Relations                            0   
              Setting          Social Relations                            0   
              Operationalizing Social Relations                            0   
              Pursuing         Social Relations                            0   

domain                                                                   \
app                                             Mobills: Budget Planner   
type          phase            feat   subfeat                             
Cross-Cutting Eliciting        Social Relations                       0   
              Defining         Social Relations                       0   
              Setting          Social Relations                       0   
              Operationalizing Social Relations                       0   
              Pursuing         Social Relations                       0   

domain                                                                          \
app                                             Money Lover - Spending Manager   
type          phase            feat   subfeat                                    
Cross-Cutting Eliciting        Social Relations                              0   
              Defining         Social Relations                              0   
              Setting          Social Relations                              0   
              Operationalizing Social Relations                              0   
              Pursuing         Social Relations                              0   

domain                                                                          \
app                                             Money Manager: Expense Tracker   
type          phase            feat   subfeat                                    
Cross-Cutting Eliciting        Social Relations                              0   
              Defining         Social Relations                              0   
              Setting          Social Relations                              0   
              Operationalizing Social Relations                              0   
              Pursuing         Social Relations                              0   

domain                                                                          \
app                                             Wallet: Budget Expense Tracker   
type          

In [64]:
# create a dataframe for domain-specific features
df_phase_specific = df_filtered.loc[df_filtered.index.get_level_values('type') == 'Phase-Specific']
df_phase_specific.head(5)

domain                                                         Finance  \
app                                           52 Weeks Money Challenge   
type           phase     feat         subfeat                            
Phase-Specific Eliciting Description                                 0   
                         Diagnosis                                   0   
                         Discovery                                   0   
                         Prediction                                  0   
                         Prescription                                0   

domain                                                                        \
app                                           Bills Reminder, Budget Planner   
type           phase     feat         subfeat                                  
Phase-Specific Eliciting Description                                       0   
                         Diagnosis                                         0   
                         Discovery                                         0   
                         Prediction                                        0   
                         Prescription                                      0   

domain                                                                      \
app                                           Fortune City - A Finance App   
type           phase     feat         subfeat                                
Phase-Specific Eliciting Description                                     0   
                         Diagnosis                                       0   
                         Discovery                                       0   
                         Prediction                                      0   
                         Prescription                                    0   

domain                                                                 \
app                                           Mobills: Budget Planner   
type           phase     feat         subfeat                           
Phase-Specific Eliciting Description                                0   
                         Diagnosis                                  0   
                         Discovery                                  0   
                         Prediction                                 0   
                         Prescription                               0   

domain                                                                        \
app                                           Money Lover - Spending Manager   
type           phase     feat         subfeat                                  
Phase-Specific Eliciting Description                                       0   
                         Diagnosis                                         0   
                         Discovery                                         0   
                         Prediction                                        0   
                         Prescription                                      0   

domain                                                                        \
app                                           Money Manager: Expense Tracker   
type           phase     feat         subfeat                                  
Phase-Specific Eliciting Description                                       0   
                         Diagnosis                                         0   
                         Discovery                                         0   
                         Prediction                                        0   
                         Prescription                                      0   

domain                                                                        \
app                                           Wallet: Budget Expense Tracker   
type           phase     feat         subfeat                                  
Phase-Specific Eliciting Descripti

In [65]:
# get all phases from df
phases = df_filtered.index.get_level_values('phase').unique()
phases

Index(['Eliciting', 'Defining', 'Setting', 'Operationalizing', 'Pursuing',
       'Tracking', 'Reflecting', 'Consequencing'],
      dtype='object', name='phase')

In [66]:
new_cols = []
for t in df_cross_cutting.T.columns.tolist():
    feat = t[3]
    subfeat = mapping.get(feat, '')
    if subfeat != feat:
        new_t = t[:3] + (subfeat, feat)
    else:
        new_t = t[:3] + ('', feat)
    new_cols.append(new_t)
new_cols[:5]

[('Cross-Cutting', 'Eliciting', 'Social', '', 'Relations'),
 ('Cross-Cutting', 'Defining', 'Social', '', 'Relations'),
 ('Cross-Cutting', 'Setting', 'Social', '', 'Relations'),
 ('Cross-Cutting', 'Operationalizing', 'Social', '', 'Relations'),
 ('Cross-Cutting', 'Pursuing', 'Social', '', 'Relations')]

In [67]:
# recreate multiindex
new_multiindex = pd.MultiIndex.from_tuples(
    new_cols,
    names=['type', 'phase', 'feat', 'middlefeat', 'subfeat']
)

# set index
df_cross_cutting.index = new_multiindex

In [68]:
# fig 17b Mean of all domains
def create_heatmap_mean_domain(df):
    feature_type = 'Cross-Cutting'
    domain_order = ['Sustainability', 'Learning', 'Wellbeing', 'Productivity', 'Finance', 'General']

    # reverse the order
    domain_order = domain_order[::-1]

    # group by domain and calculate the mean
    df_grouped_phase = df.T.groupby(level='domain').mean().T.groupby(level=['feat', 'middlefeat', 'subfeat']).mean().T

    # reorder based on domain order
    df_grouped_phase = df_grouped_phase.reindex(domain_order)

    # convert to 2D array for heatmap
    heatmap_data = df_grouped_phase.values

    # Compute row and column averages
    row_averages = np.mean(heatmap_data, axis=1, keepdims=True)
    col_averages = np.mean(heatmap_data, axis=0, keepdims=True)
    overall_average = np.mean(heatmap_data)

    # Append row averages to heatmap_data
    heatmap_data_with_row_avg = np.hstack([heatmap_data, row_averages])

    # Create new row with column averages and overall average
    new_row = np.append(col_averages, overall_average).reshape(1, -1)

    # Append new row to data
    heatmap_data_final = np.vstack([new_row, heatmap_data_with_row_avg])

    # Add average to the front of the domain order
    domain_order = ['Average'] + domain_order

    # Prepare the axis labels
    feat_subfeat_list = [
        [' ', 'Diversification', 'Diversification', 'Enrichment', 'Enrichment', 'Derivation', 'Derivation', '\u2062',
         '\u2062', '   '],
        ['Integration', 'Adaptation', 'Recommendation', 'Profile', 'Context',
         'Interpretation of<br>User Behaviour', 'Explanation of<br>System Behaviour',
         'Relation',
         'Interaction', 'Average']]

    # Create the heatmap
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data_final,  # Updated values for the heatmap
        x=feat_subfeat_list,  # X-axis labels (Features/Subfeatures)
        y=[x + ' ' for x in domain_order],  # Y-axis labels (Domains)
        colorscale='Inferno_r',
        zmin=0, zmax=1, xgap=2, ygap=2,
        colorbar=dict(
            tickformat='.0%'  # Set formatting to percentages
        ),
        texttemplate="%{z:.0%}",  #texttemplate="%{text}",
    ))

    # Anpassen des Layouts
    fig.update_layout(
        template='plotly_white', height=800, width=800,
        # make x label 90 degree
        xaxis_tickangle=90,
        # make xaxis title bold
        xaxis_title_font=dict(weight='bold')
    )

    # make font bold
    fig.update_xaxes(tickfont=dict(family='Arial', size=13, color='black', weight='bold'))
    fig.update_yaxes(tickfont=dict(family='Arial', size=13, color='black', weight='bold'))
    
    # get number of rows and columns for placing lines in the heatmap
    n_rows, n_cols = heatmap_data.shape
    n_rows = n_rows + 1

    # Add vertical lines between all cells for left/right borders
    for i in range(n_cols + 2):
        feat_labels = feat_subfeat_list[0]
        if i == n_cols: # bold line for mean
            fig.add_shape(
                type="line",
                x0=i - 0.5, y0=-0.5, x1=i - 0.5, y1=n_rows - 0.5,
                line=dict(color="black", width=3),
            )
        elif i < n_cols and i != 0: # line between feature-classes
            if feat_labels[i - 1] != feat_labels[i]:
                fig.add_shape(
                    type="line",
                    x0=i - 0.5, y0=-0.5, x1=i - 0.5, y1=n_rows - 0.5,
                    line=dict(color="black", width=1.5),
                )
        else: # first and last line
            fig.add_shape(
                type="line",
                x0=i - 0.5, y0=-0.5, x1=i - 0.5, y1=n_rows - 0.5,
                line=dict(color="black", width=1.5),
            )
    
    # add one horizontal line for the mean
    fig.add_shape(
        type="line",
        x0=-0.5, y0=1 - 0.5, x1=n_cols + 1 - 0.5, y1=1 - 0.5,
        line=dict(color="black", width=3),
    )
    
    # add annotation for the overall classes
    fig.add_annotation(
        x=0.5,  # Relative x-position
        y=-0.307,  # Position above the x-axis
        text="|\u2009               System                |                    Knowledge                    |\u200A          Social        \u200A|              |",
        showarrow=False,
        xref="paper",
        yref="paper",
        font=dict(size=14, color="black", family="Arial", weight="bold"),
        align="center",
    )
    
    # show image
    fig.show()

    # safe image as pdf and png
    fig.write_image(f'../images/pdf/17B_{feature_type}_Features_Distribution_in_all_domains_new_heatmap.pdf')
    fig.write_image(f'../images/png/17B_{feature_type}_Features_Distribution_in_all_domains_new_heatmap.png')


create_heatmap_mean_domain(df_cross_cutting)

In [69]:
# 17a final
def create_heatmap_mean_phase(df, order_dict):
    # define feature type and phase order
    feature_type = 'Cross-Cutting'
    phase_order = ['Eliciting', 'Defining', 'Setting', 'Operationalizing', 'Pursuing', 'Tracking', 'Consequencing',
                   'Reflecting']

    # Group by Phase, Domain, Feature und Subfeature and calculate the mean
    df_grouped_phase = df.groupby(level=['phase', 'feat', 'subfeat']).sum().T.groupby(level='domain').mean().T.unstack(
        level='phase')
    df_grouped_phase = df_grouped_phase.T.groupby(level=['phase']).mean()

    # Funktion for extracting the ordered feature-subfeature pairs
    def get_ordered_feats(order_dict):
        ordered_feats = []
        for feat, subfeats in order_dict.items():
            if subfeats:
                for subfeat in subfeats:
                    ordered_feats.append((feat, subfeat))
            else:
                ordered_feats.append((feat, None))
        return ordered_feats

    ordered_feats = get_ordered_feats(order_dict)

    # reorder the columns of df_grouped_phase according to the desired order
    df_columns = list(df_grouped_phase.columns)
    ordered_columns = [col for col in ordered_feats if col in df_columns]

    df_grouped_phase = df_grouped_phase[ordered_columns]
    df_grouped_phase = df_grouped_phase.T[phase_order[::-1]].T

    # convert to 2D array for heatmap
    heatmap_data = df_grouped_phase.values

    # Compute row and column averages
    row_averages = np.mean(heatmap_data, axis=1, keepdims=True)
    col_averages = np.mean(heatmap_data, axis=0, keepdims=True)
    overall_average = np.mean(heatmap_data)

    # Append row averages to heatmap_data
    heatmap_data_with_row_avg = np.hstack([heatmap_data, row_averages])

    # Create new row with column averages and overall average
    new_row = np.append(col_averages, overall_average).reshape(1, -1)

    # Append new row to data
    heatmap_data_final = np.vstack([new_row, heatmap_data_with_row_avg])

    # Add average to the front of the domain order
    phase_order = ['Average'] + phase_order[::-1]

    # Prepare the axis labels
    feat_subfeat_list = [
        [' ', 'Diversification', 'Diversification', 'Enrichment', 'Enrichment', 'Derivation', 'Derivation',
         '\u2062',
         '\u2062', '   '],
        ['Integration', 'Adaptation', 'Recommendation', 'Profile', 'Context',
         'Interpretation of<br>User Behaviour', 'Explanation of<br>System Behaviour',
         'Relation',
         'Interaction', 'Average']]
    
    # Create the heatmap
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data_final,  # Updated values for the heatmap
        x=feat_subfeat_list,  # X-axis labels (Features/Subfeatures)
        y=[x + ' ' for x in phase_order],  # Y-axis labels (Domains)
        colorscale='Inferno_r',
        zmin=0, zmax=1, xgap=2, ygap=2,
        colorbar=dict(
            tickformat='.0%'  # Set formatting to percentages
        ),
        texttemplate="%{z:.0%}"
    ))

    # set layout
    fig.update_layout(
        template='plotly_white', height=800, width=800,
        xaxis_tickangle=90,
        xaxis_title_font=dict(weight='bold', color='black')
    )

    # make font bold
    fig.update_xaxes(tickfont=dict(family='Arial', size=13, color='black', weight='bold'))
    fig.update_yaxes(tickfont=dict(family='Arial', size=13, color='black', weight='bold'))
    
    # get number of rows and columns for placing lines in the heatmap
    n_rows, n_cols = heatmap_data.shape
    n_rows = n_rows + 1

    # Add vertical lines between all cells for left/right borders
    for i in range(n_cols + 2):  # evtl +1
        feat_labels = feat_subfeat_list[0]
        if i == n_cols: # bold line for mean
            fig.add_shape(
                type="line",
                x0=i - 0.5, y0=-0.5, x1=i - 0.5, y1=n_rows - 0.5,
                line=dict(color="black", width=3),
            )
        elif i < n_cols and i != 0: # line between feature-classes
            if feat_labels[i - 1] != feat_labels[i]:
                fig.add_shape(
                    type="line",
                    x0=i - 0.5, y0=-0.5, x1=i - 0.5, y1=n_rows - 0.5,
                    line=dict(color="black", width=1.5),
                )
        else: # first and last line
            fig.add_shape(
                type="line",
                x0=i - 0.5, y0=-0.5, x1=i - 0.5, y1=n_rows - 0.5,
                line=dict(color="black", width=1.5),
            )
    
    # add one horizontal line for the mean
    fig.add_shape(
                type="line",
                x0=-0.5, y0=1 - 0.5, x1=n_cols + 1 - 0.5, y1=1 - 0.5,
                line=dict(color="black", width=3),
            )
    
    # add annotation for the overall classes
    fig.add_annotation(
        x=0.5,  # Relative x-position
        y=-0.307,  # Position above the x-axis
        text="\u2009|               System               \u200A|                   Knowledge                   |\u200A          Social        |              |",
        showarrow=False,
        xref="paper",
        yref="paper",
        font=dict(size=14, color="black", family="Arial", weight="bold"),
        align="center",
    )
    # show image/plot
    fig.show()

    # save as pdf and png
    fig.write_image(f'../images/pdf/17A_{feature_type}_Features_Distribution_in_all_phases_heatmap.pdf')
    fig.write_image(f'../images/png/17A_{feature_type}_Features_Distribution_in_all_phases_heatmap.png')

create_heatmap_mean_phase(df_cross_cutting, cross_cutting_order)

In [70]:
def create_heatmap_each_phase(df, phase, feature_type, order_dict):
    # define the desired order of the domains
    domain_order = ['Sustainability', 'Learning', 'Wellbeing', 'Productivity', 'Finance', 'General']
    
    # reverse the order
    domain_order = domain_order[::-1]
    
    # filter by phase, group by feat and subfeat and domain and calculate the mean
    df_grouped_phase = df[df.index.get_level_values('phase') == phase].groupby(
        level=['feat', 'subfeat']).sum().T.groupby(level='domain').mean()

    # Reorder the domains according to the desired order
    df_grouped_phase = df_grouped_phase.reindex(domain_order)

    # Function to extract the ordered feature-subfeature pairs based on the order_dict
    def get_ordered_feats(order_dict):
        ordered_feats = []
        for feat, subfeats in order_dict.items():
            if subfeats:
                for subfeat in subfeats:
                    ordered_feats.append((feat, subfeat))
            else:
                ordered_feats.append((feat, None))
        return ordered_feats

    ordered_feats = get_ordered_feats(order_dict)

    # Reorder the columns of df_grouped_phase according to the desired order
    df_columns = list(df_grouped_phase.columns)
    ordered_columns = [col for col in ordered_feats if col in df_columns]
    df_grouped_phase = df_grouped_phase[ordered_columns]

    # Convert the data into a 2D array form for the heatmap
    heatmap_data = df_grouped_phase.values

    # Compute row and column averages
    row_averages = np.mean(heatmap_data, axis=1, keepdims=True) 
    col_averages = np.mean(heatmap_data, axis=0)
    overall_average = np.mean(heatmap_data)

    # Append row averages to heatmap_data (neue Spalte)
    heatmap_data_with_row_avg = np.hstack([heatmap_data, row_averages])

    # Append new row with column averages and overall average
    new_row = np.append(col_averages, overall_average).reshape(1, -1)  # Neue Zeile
    heatmap_data_final = np.vstack([new_row, heatmap_data_with_row_avg])  # Zeilen anhängen

    # append average into the front of the domain order
    domain_order = ['Average'] + domain_order
    
    
    # Prepare the axis labels
    feat_labels = []
    sub_feat_labels = []
    empty_feat = ''
    
    # rename the features and subfeatures
    for feat, sub_feat in df_grouped_phase.columns:
        if sub_feat == "Documenting":
            sub_feat = "Growth"

        if "goal" in sub_feat:
            sub_feat = sub_feat.replace(" goal", "")
        elif "goals" in sub_feat:
            sub_feat = sub_feat.replace(" goals", "")

        if len(sub_feat) > 1:
            if sub_feat[-1] == "s":
                sub_feat = sub_feat[:-1]

        if sub_feat == "Hierarchie":
            sub_feat = "Hierarchy"

        if sub_feat == "Goal specific reminder":
            sub_feat = "Goal"
        elif sub_feat == "Process specific reminder":
            sub_feat = "Process"
        elif sub_feat == "Adjustability":
            sub_feat = "Adjustability of Tracking Data"
        elif sub_feat == "Verification":
            sub_feat = "Verification of Tracking Data"
        elif sub_feat == "One time":
            sub_feat = "One-time"
        elif "time" in sub_feat:
            sub_feat = sub_feat.replace("time", "Time")
        elif "day" in sub_feat:
            sub_feat = sub_feat.replace("day", "Day")
        elif sub_feat == "Pre-defined event":
            sub_feat = "Pre-defined Event"
        elif sub_feat == "Custom event":
            sub_feat = "Custom Event"
        elif sub_feat == "Skip if completed":
            sub_feat = "Skip if Completed"
        elif sub_feat == "Different alert types":
            sub_feat = "Different Alert Type"
        elif sub_feat == "Different alert type":
            sub_feat = "Different Alert Type"
        elif subfeat == "Snooze reminder":
            sub_feat = "Snooze Reminder"

        if "pursuing" in sub_feat.lower():
            sub_feat = "Additional Features"

        if "pursuing" in feat.lower():
            feat = "Additional Features"

        if feat == "Abstraction Level":
            feat = "Level"
        elif feat == "Goal Baseline":
            feat = "Baseline"
        elif feat == "Recurrence":
            feat = "Recur-<br>rence"
        elif feat == "Event-based reminder":
            feat = "Event-based <br> Reminder"
        elif feat == "Context of reminder":
            feat = "Context of<br>Reminder"
        elif feat == "Customization of reminder":
            feat = "Customization of<br>Reminder"
        elif feat == "Tracking":
            feat = "Tracking Mode"
        elif feat == "Temporality":
            feat = "Time"
        elif feat == "Granularity":
            feat = "Granularity"

        if feat == "Constraints":
            feat = "Constraint"

        if feat == "Interdependencies":
            feat = "Interdependency"

        if "reminder" in feat:
            feat = feat.replace("reminder", "Reminder")

        if "reminder" in sub_feat:
            sub_feat = sub_feat.replace("reminder", "Reminder")

        if feat and sub_feat == '':
            empty_feat = empty_feat + ' '
            sub_feat = feat
            feat = empty_feat
        feat_labels.append(feat)
        sub_feat_labels.append(sub_feat if pd.notna(sub_feat) else '')

    # Update labels
    feat_labels.append("\u2062")
    sub_feat_labels.append('Average')

    feat_subfeat_list = [feat_labels, sub_feat_labels]
    
    # if feature type is cross-cutting use this feature-subfeature list
    if feature_type == 'Cross-Cutting':
        feat_subfeat_list = [
            [' ', 'Diversification', 'Diversification', 'Enrichment', 'Enrichment', 'Derivation', 'Derivation', '  ',
             '  ', '   '], ['Integration', 'Adaptation', 'Recommendation', 'Profile', 'Context',
                            'Interpretation of<br>User Behaviour', 'Explanation of<br>System Behaviour',
                            'Relation',
                            'Interaction', 'Average']]
    
    # Create the heatmap
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data_final,  # Updated values for the heatmap
        x=feat_subfeat_list,  # X-axis labels (Features/Subfeatures)
        y=[x + ' ' for x in domain_order],  # Y-axis labels (Domains)
        colorscale='Inferno_r',
        zmin=0, zmax=1, xgap=2, ygap=2,
        colorbar=dict(
            tickformat='.0%'  # Set formatting to percentages
        ),
        texttemplate="%{z:.0%}",  #texttemplate="%{text}",
    ))
    
    # Adjusting the layout
    fig.update_layout(
        template='plotly_white', height=900, width=800,
        xaxis_tickangle=90,
        xaxis=dict(titlefont=dict(weight='bold', color='black'))
    )
    
    # make font bold
    fig.update_xaxes(tickfont=dict(family='Arial', size=13, color='black', weight='bold'))
    fig.update_yaxes(tickfont=dict(family='Arial', size=13, color='black', weight='bold'))

    # Adding vertical lines to set off categories and create left/right borders
    n_rows, n_cols = heatmap_data.shape
    n_cols += 1

    # Add vertical lines between groups
    for i in range(n_cols + 1):  # evtl +1
        if feature_type == "Cross-Cutting":
            feat_labels = feat_subfeat_list[0]
        if i == n_cols - 1: # bold line for mean
            fig.add_shape(
                type="line",
                x0=i - 0.5, y0=-0.5, x1=i - 0.5, y1=n_rows + 1 - 0.5,
                line=dict(color="black", width=3),
            )
        elif i < n_cols - 1 and i != 0: # line between feature-classes
            if not feat_labels[i - 1] == feat_labels[i]:
                fig.add_shape(
                    type="line",
                    x0=i - 0.5, y0=-0.5, x1=i - 0.5, y1=n_rows + 1 - 0.5,
                    line=dict(color="black", width=1.5),
                )
        else: # first and last line
            fig.add_shape(
                type="line",
                x0=i - 0.5, y0=-0.5, x1=i - 0.5, y1=n_rows + 1 - 0.5,
                line=dict(color="black", width=1.5),
            )
        
        # Add horizontal line for the mean
        fig.add_shape(
            type="line",
            x0=-0.5, y0=1 - 0.5, x1=n_cols - 0.5, y1=1 - 0.5,
            line=dict(color="black", width=3),
        )
    
    fig.update_layout(margin=dict(b=180))

    # Add annotation for the overall classes if feature type is cross-cutting
    if feature_type == "Cross-Cutting":
        fig.add_annotation(
            x=0.5,  # Relative x-position
            y=-0.25,  # Position above the x-axis
            text="|\u2009               System                |                    Knowledge                    |\u200A          Social        \u200A|              |",
            showarrow=False,
            xref="paper",
            yref="paper",
            font=dict(size=14, color="black", family="Arial", weight="bold"),
            align="center",
        )

    # Show the figure
    fig.show()

    # Save the plot
    fig.write_image(f'../images/pdf/{feature_type}_Features_Distribution_in_{phase}_heatmap.pdf')
    fig.write_image(f'../images/png/{feature_type}_Features_Distribution_in_{phase}_heatmap.png')

# create the heatmap for each phase
for phase in phases:
    create_heatmap_each_phase(df_phase_specific, phase, 'Phase-Specific', feature_order.get(phase, {}))
    create_heatmap_each_phase(df_cross_cutting, phase, 'Cross-Cutting', cross_cutting_order)